In [1]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Operator

In [2]:
def ket(x: list):
    return np.array(x).reshape(-1, 1).astype(np.complex128)


state00 = ket([1, 0, 0, 0])
state11 = ket([0, 0, 0, 1])

phi_I = (state00 + state11) / np.sqrt(2)
phi_X = np.dot(np.kron(np.eye(2), np.array([[0, 1], [1, 0]])), phi_I)
phi_Z = np.dot(np.kron(np.eye(2), np.array([[1, 0], [0, -1]])), phi_I)
phi_XZ = np.dot(
    np.kron(np.eye(2), np.dot(np.array([[0, 1], [1, 0]]), np.array([[1, 0], [0, -1]]))),
    phi_I,
)

In [3]:
def project_itself(x: np.matrix):
    return np.dot(x, x.reshape(1, -1))


# take a smallest example, 2 qubit and 1 2-qubit gate
# total used qubits are 2Dn + n qubts, latter n for input
n = 2
D = 1
total = n * (2 * D + 1)



In [4]:
# pick a perturbation "epsilon"
eps = 0.1
lamb = eps * project_itself(phi_I) + (
    project_itself(phi_X) + project_itself(phi_Z) + project_itself(phi_XZ)
)


def lamb_n(n: int):
    res = lamb
    for _ in range(n - 1):
        res = np.kron(res, lamb)
    return res


# penalize on states that have |1>
penalize_one = np.diag([0, 1, 1, 2])
# extend to 2n qubits that fits the injective map
# these qubits are the next column
penalize_one = np.kron(penalize_one, np.eye(4))
# multiply with injective map, recall that n=2
H_in = np.dot(np.dot(lamb_n(2), penalize_one), lamb_n(2))
# finally extend to actual size of Hamiltonian
# it is known that the input is at the beginning
# so there are one column / 2 qubits at the end
H_in = np.kron(H_in, np.eye(4))


# check output, but now the extension is at the beginning
H_out = np.kron(np.eye(4), penalize_one)


# since we have smallest example, we use the boundary case
X = np.array([[0, 1], [1, 0]])
phi_U = np.dot(np.kron(np.eye(4), np.kron(X, X)), np.kron(phi_I, phi_I))
penalize_U = np.eye(16) - project_itself(phi_U)
# note that here is no more lamb_4, because it is boundary -> lamb_2
H_u = np.dot(np.dot(lamb_n(2), penalize_U), lamb_n(2))
# based on the image, the middle part is blue wavy line
H_u = np.kron(np.eye(4), H_u)


In [5]:
from frontend import AdiabaticProgram
from backend.cpu import CPUBackend
from scipy import sparse as sp

class TempAP(AdiabaticProgram):
    def __init__(self) -> None:
        super().__init__(H_in, H_in + H_out + H_u, 2**6, 6, 6)
    def compile(self):
        return sp.csc_matrix(self.H_init), sp.csc_matrix(self.H_final)
    
CPUBackend().run(TempAP(), 1024)

100%|██████████| 7/7 [00:00<00:00, 35.24it/s]


{'111100': 87,
 '001100': 62,
 '111111': 11,
 '110011': 197,
 '110000': 124,
 '000000': 364,
 '001111': 9,
 '000011': 170}